In [1]:
import os
import coiled
import numpy as np
import pandas as pd

from dask.distributed import wait
from dask.distributed import print as log

from pipeline.plotting import make_plot
from pipeline.complexity import is_complex
from pipeline.lightcurves import make_lightcurves, load_lc

In [2]:
targets = pd.read_csv("./misc/targets.txt")

In [3]:
def process_target(target):
    try:
        os.mkdir("./lightcurves/")
    except:
        pass
    
    tic, ra, dec = target
    tic = f"TIC {tic}"

    results = []
    sectors = make_lightcurves(tic=tic, ra=ra, dec=dec)
    
    for sector in sectors: 
        lc = load_lc(sector[0], sector[1])
        try:
            results.append((sector[0], sector[1], is_complex(lc)))
        except:
            results.append((sector[0], sector[1], pd.NA))

    return results

In [4]:
cluster = coiled.Cluster(
    n_workers=3,
    region="us-east-1",
    name="pipeline_test",
)

client = cluster.get_client()

In [5]:
%%time
target_tuples = [tuple(target) for target in list(targets.itertuples(index=False))]
futures = []

for target in target_tuples:
    future = client.submit(process_target, target)
    futures.append(future)

results = client.gather(futures)

CPU times: user 165 ms, sys: 26.9 ms, total: 192 ms
Wall time: 33.3 s


In [7]:
cluster.close()

In [8]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

%matplotlib widget

results_flat = []
for result in results:
    results_flat.extend(result)

output = widgets.Output()
counter = 0

def plot(i):
    tic, sector, complexity = results_flat[i]

    if complexity not in [True, False]:
        complexity = None

    with output:
        clear_output(wait=True)
        make_plot(load_lc(tic, sector), complexity=complexity)
        plt.show()

def on_button_click(b):
    """Callback function for button press"""
    global counter
    counter += 1  # Increment counter
    plot(counter)

button = widgets.Button(description="Next Graph")
button.on_click(on_button_click)

display(button, output)
plot(counter)


Button(description='Next Graph', style=ButtonStyle())

Output()

In [52]:
results_flat

[('TIC 425933644', 1, True),
 ('TIC 425933644', 2, True),
 ('TIC 425933644', 28, True),
 ('TIC 425933644', 28, True),
 ('TIC 425933644', 29, True),
 ('TIC 425933644', 68, True),
 ('TIC 425933644', 69, True),
 ('TIC 406040223', 11, False),
 ('TIC 406040223', 37, False),
 ('TIC 406040223', 38, False),
 ('TIC 406040223', 64, False),
 ('TIC 406040223', 65, <NA>),
 ('TIC 448002486', 11, False),
 ('TIC 448002486', 38, False),
 ('TIC 448002486', 64, False),
 ('TIC 448002486', 65, False)]